In [119]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

# Downloading and return Calculations

In [120]:
def returns(ticker_list):
    for ticker in tqdm(ticker_list):
        df = yf.download(ticker,start = '2020-01-01')
        df['daily_ret'] = df['Adj Close'].pct_change()
        df_1[ticker] = df['daily_ret']
        df_1.dropna(inplace = True)
    return df_1
ticker_list = ['AAPL','TSLA','AMZN','GOOGL','META']
n = len(ticker_list)
df_1 = pd.DataFrame()
df = returns(ticker_list)
df

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
100%|██████████| 5/5 [00:01<00:00,  4.42it/s]


,AAPL,TSLA,AMZN,GOOGL,META
Date,,,,,
2020-01-03,-0.009722,0.029633,-0.012139,-0.005231,-0.005291
2020-01-06,0.007968,0.019255,0.014886,0.026654,0.018834
2020-01-07,-0.004703,0.038801,0.002092,-0.001932,0.002164
2020-01-08,0.016086,0.049205,-0.007809,0.007118,0.010138
2020-01-09,0.021241,-0.021945,0.004799,0.010498,0.014311
...,...,...,...,...,...
2024-06-10,-0.019148,-0.020791,0.014976,0.004304,0.019555
2024-06-11,0.072649,-0.018010,0.000909,0.009199,0.009690
2024-06-12,0.028578,0.038849,-0.001816,0.006624,0.002700


# Covariance matrix 

In [107]:
sigma = df.cov()
Sigma = sigma.values
Sigma

array([[0.00042683, 0.00043257, 0.00028787, 0.00028619, 0.00034465],
       [0.00043257, 0.00177181, 0.00041929, 0.00034747, 0.00041123],
       [0.00028787, 0.00041929, 0.00053081, 0.000314  , 0.00041456],
       [0.00028619, 0.00034747, 0.000314  , 0.00043457, 0.00039493],
       [0.00034465, 0.00041123, 0.00041456, 0.00039493, 0.00085607]])

# SQP 

In [115]:
def portfolio_volatility(x, Sigma):
    return np.sqrt(x.T @ Sigma @ x)

def calculate_mcr(x, Sigma):
    portfolio_vol = portfolio_volatility(x, Sigma)
    mcr = (Sigma @ x) / portfolio_vol
    return mcr
    
def objective(x, Sigma):
    sum_squared_diff = 0
    MCR = calculate_mcr(x, Sigma) 
    for i in range(len(x)):
        for j in range(len(x)):
            diff = x[i] * MCR[i] - x[j] * MCR[j]
            sum_squared_diff += diff ** 2
    return sum_squared_diff


def constraint_eq(x):
    return np.sum(x) - 1

bounds = [(0, 1) for _ in range(n)]

x0 = np.ones(n) / n  

constraints = {'type': 'eq', 'fun': constraint_eq}

solution = minimize(objective, x0, args=(Sigma,), method='SLSQP', bounds=bounds, constraints=constraints)

print('Optimal solution:', solution.x)
print('Objective function value:', solution.fun)


Optimal solution: [0.21925696 0.14203204 0.21581026 0.22047782 0.20242292]
Objective function value: 6.5819183768814185e-06
